In [2]:
# import required libriaries
from pathlib import Path
from helpers import *

In [3]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [4]:
# read in latest training set
df_train = pd.read_csv(path/'train_v3.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [5]:
df_train.tail(50)

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
90387,Luke_Freeman,38,3,0,Sheffield United,Southampton,0.348678,0.526987,False,0,...,1,3,0.0,-30,8,38,0,2020-07-26T15:00:00Z,1920,0.25
90388,Kelland_Watts,38,2,16,Newcastle United,Liverpool,0.611449,2.406333,True,1,...,3,1,0.0,3319,4336,1017,0,2020-07-26T15:00:00Z,1920,1.00
90389,Shkodran_Mustafi,38,2,0,Arsenal,Watford,1.452343,0.472917,True,0,...,2,3,0.0,-1929,135,2064,0,2020-07-26T15:00:00Z,1920,0.00
90390,Mace_Goodridge,38,3,0,Burnley,Brighton and Hove Albion,0.370648,0.541184,True,0,...,2,1,0.0,1,26,25,0,2020-07-26T15:00:00Z,1920,1.00
90391,Dan_Gosling,38,3,90,Bournemouth,Everton,0.565681,1.005607,False,2,...,3,1,1.0,-87,95,182,0,2020-07-26T15:00:00Z,1920,1.00
90392,Tammy_Abraham,38,4,12,Chelsea,Wolverhampton Wanderers,1.999543,0.811175,True,1,...,0,2,11.0,-10050,10886,20936,0,2020-07-26T15:00:00Z,1920,1.00
90393,Nicolás_Otamendi,38,2,0,Manchester City,Norwich,2.430397,0.327574,True,0,...,0,5,0.0,-1574,7965,9539,0,2020-07-26T15:00:00Z,1920,1.00
90394,James_Daly,38,4,0,Crystal Palace,Tottenham Hotspur,0.430493,1.604904,True,0,...,1,1,0.0,125,165,40,0,2020-07-26T15:00:00Z,1920,0.00
90395,Declan_Rice,38,3,90,West Ham United,Aston Villa,0.709989,0.553818,True,5,...,1,1,3.0,15208,18372,3164,0,2020-07-26T15:00:00Z,1920,1.00
90396,Connor_Wickham,38,4,0,Crystal Palace,Tottenham Hotspur,0.430493,1.604904,True,0,...,1,1,0.0,1567,2425,858,0,2020-07-26T15:00:00Z,1920,0.00


In [6]:
# df_train[(df_train['gw']==24) & (df_train['season']=='1920') & (df_train['team']=='Liverpool')]

In [7]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

39

In [7]:
# last_gw = 28

In [8]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [9]:
df_latest.drop_duplicates(inplace=True)

In [10]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Chris_Mepham,38,2,0,Bournemouth,Everton,0.565681,1.005607,False,0,...,45168,3,1,0.0,-238,13,251,0,2020-07-26T15:00:00Z,1920
1,Paulo_Gazzaniga,38,1,0,Tottenham Hotspur,Crystal Palace,1.604904,0.430493,False,0,...,113721,1,1,0.0,-1374,1716,3090,0,2020-07-26T15:00:00Z,1920
2,John_Egan,38,2,90,Sheffield United,Southampton,0.348678,0.526987,False,1,...,518244,1,3,0.0,2202,31171,28969,0,2020-07-26T15:00:00Z,1920
3,Max_Meyer,38,3,0,Crystal Palace,Tottenham Hotspur,0.430493,1.604904,True,0,...,5985,1,1,0.0,-8,14,22,0,2020-07-26T15:00:00Z,1920
4,Fabian_Delph,38,2,0,Everton,Bournemouth,1.005607,0.565681,True,0,...,12733,3,1,0.0,-51,22,73,0,2020-07-26T15:00:00Z,1920
5,James_Justin,38,2,90,Leicester City,Manchester United,1.076834,1.753447,True,1,...,38893,2,0,0.0,-5516,908,6424,0,2020-07-26T15:00:00Z,1920
6,Che_Adams,38,4,84,Southampton,Sheffield United,0.526987,0.348678,True,13,...,31051,1,3,64.0,3348,4349,1001,0,2020-07-26T15:00:00Z,1920
7,Mahmoud Ahmed_Ibrahim Hassan,38,3,89,Aston Villa,West Ham United,0.553818,0.709989,False,2,...,153177,1,1,11.0,47220,50470,3250,0,2020-07-26T15:00:00Z,1920
8,Tyreece_John-Jules,38,4,0,Arsenal,Watford,1.452343,0.472917,True,0,...,521,2,3,0.0,33,138,105,0,2020-07-26T15:00:00Z,1920
9,Adrian_Mariappa,38,2,45,Watford,Arsenal,0.472917,1.452343,False,1,...,18383,2,3,0.0,-49,377,426,0,2020-07-26T15:00:00Z,1920


In [11]:
df_latest.shape

(666, 34)

In [12]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [13]:
last_play_proba.head()

,player,play_proba
0,Shkodran_Mustafi,0.0
1,Héctor_Bellerín,1.0
2,Sead_Kolasinac,1.0
3,Ainsley_Maitland-Niles,1.0
4,Sokratis_Papastathopoulos,1.0


In [14]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [15]:
df_latest.drop_duplicates(inplace=True)

In [16]:
df_train.drop_duplicates(inplace=True)

In [17]:
df_train.shape

(89771, 35)

In [18]:
df_latest.shape

(666, 35)

In [19]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [20]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
90432,Tommy_Doyle,38,3,0,Manchester City,Norwich,2.430397,0.327574,True,0,...,0,5,0.0,-2,22,24,0,2020-07-26T15:00:00Z,1920,1.0
90433,Joseph_Anang,38,1,0,West Ham United,Aston Villa,0.709989,0.553818,True,0,...,1,1,0.0,70,270,200,0,2020-07-26T15:00:00Z,1920,1.0
90434,Erik_Pieters,38,2,90,Burnley,Brighton and Hove Albion,0.370648,0.541184,True,3,...,2,1,2.0,139816,144388,4572,1,2020-07-26T15:00:00Z,1920,1.0
90435,Japhet_Tanganga,38,2,0,Tottenham Hotspur,Crystal Palace,1.604904,0.430493,False,0,...,1,1,0.0,7999,14840,6841,0,2020-07-26T15:00:00Z,1920,1.0
90436,Ravel_Morrison,38,3,0,Sheffield United,Southampton,0.348678,0.526987,False,0,...,1,3,0.0,-23,7,30,0,2020-07-26T15:00:00Z,1920,0.0


In [21]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train_v3.csv')

In [22]:
# df_train_new = pd.read_csv(path/'train_v3.csv', index_col=0, dtype={'season':str})

In [23]:
# df_train_new[(df_train_new['team'] == 'Manchester United') & 
#              (df_train_new['gw'] == 32) & 
#              (df_train_new['season'] == '1920')]

In [24]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [25]:
fixtures.head(20)

,home_team,away_team,gw
370,Arsenal,Watford,38
371,Burnley,Brighton and Hove Albion,38
372,Chelsea,Wolverhampton Wanderers,38
373,Crystal Palace,Tottenham Hotspur,38
374,Everton,Bournemouth,38
375,Leicester City,Manchester United,38
376,Manchester City,Norwich,38
377,Newcastle United,Liverpool,38
378,Southampton,Sheffield United,38
379,West Ham United,Aston Villa,38


In [26]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [27]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,38,Arsenal,Watford,3,57
1,38,Burnley,Brighton and Hove Albion,90,36
2,38,Chelsea,Wolverhampton Wanderers,8,39
3,38,Crystal Palace,Tottenham Hotspur,31,6
4,38,Everton,Bournemouth,11,91


In [28]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [29]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [30]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,38,Watford,Arsenal,José_Holebas,2,46,0,False
1,38,Watford,Arsenal,Christian_Kabasele,2,43,100,False
2,38,Watford,Arsenal,Adam_Masina,2,43,100,False
3,38,Watford,Arsenal,Adrian_Mariappa,2,41,100,False
4,38,Watford,Arsenal,Francisco_Femenía Far,2,42,100,False


In [31]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,38,Arsenal,Watford,Shkodran_Mustafi,2,51,0,True
1,38,Arsenal,Watford,Héctor_Bellerín,2,52,100,True
2,38,Arsenal,Watford,Sead_Kolasinac,2,52,100,True
3,38,Arsenal,Watford,Ainsley_Maitland-Niles,2,45,100,True
4,38,Arsenal,Watford,Sokratis_Papastathopoulos,2,48,100,True


In [32]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 29.6,
 'Aaron_Cresswell': 86.8,
 'Aaron_Mooy': 53.6,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 76.0,
 'Abdoulaye_Doucouré': 84.4,
 'Adalberto_Peñaranda': 0.0,
 'Adam_Idah': 14.6,
 'Adam_Lallana': 0.0,
 'Adam_Masina': 50.8,
 'Adam_Smith': 69.0,
 'Adam_Webster': 90.0,
 'Adama_Traoré': 71.6,
 'Adrian_Mariappa': 45.0,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 40.8,
 'Ainsley_Maitland-Niles': 9.8,
 'Albian_Ajeti': 0.8,
 'Alex_Iwobi': 35.25,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 40.0,
 'Alexander_Tettey': 83.75,
 'Alexandre Nascimento_Costa Silva': 0.0,
 'Alexandre_Jankewitz': 0.0,
 'Alexandre_Lacazette': 62.8,
 'Alexis_Mac Allister': 25.4,
 'Alfie_Lewis': 0.0,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 4.0,
 'Alisson_Ramses Becker': 90.0,
 'Allan_Saint-Maximin': 52.666666666666664,
 'Andre_Gray': 4.8,
 'Andreas_Christensen': 45.0,
 'Andreas_Pereira': 0.0,
 'Andrew_Robertson': 81.0,
 'Andrew_Surman': 0.0,
 'Andriy_Yarmolenko': 18

In [33]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [34]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,38,Arsenal,Watford,Shkodran_Mustafi,2,5.1,0.0,True,1.496432,0.504811,1920,0.000000
1,38,Arsenal,Watford,Héctor_Bellerín,2,5.2,1.0,True,1.496432,0.504811,1920,37.400000
2,38,Arsenal,Watford,Sead_Kolasinac,2,5.2,1.0,True,1.496432,0.504811,1920,71.600000
3,38,Arsenal,Watford,Ainsley_Maitland-Niles,2,4.5,1.0,True,1.496432,0.504811,1920,9.800000
4,38,Arsenal,Watford,Sokratis_Papastathopoulos,2,4.8,1.0,True,1.496432,0.504811,1920,0.000000
5,38,Arsenal,Watford,Nacho_Monreal,2,5.0,0.0,True,1.496432,0.504811,1920,0.000000
6,38,Arsenal,Watford,Laurent_Koscielny,2,5.0,0.0,True,1.496432,0.504811,1920,0.000000
7,38,Arsenal,Watford,Konstantinos_Mavropanos,2,4.3,0.0,True,1.496432,0.504811,1920,0.000000
8,38,Arsenal,Watford,Carl_Jenkinson,2,4.5,0.0,True,1.496432,0.504811,1920,0.000000
9,38,Arsenal,Watford,Rob_Holding,2,4.4,1.0,True,1.496432,0.504811,1920,36.000000


In [35]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')